In [9]:
import pandas as pd
import selenium
import webdriver_manager
from datetime import datetime 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support import expected_conditions as EC 
from time import sleep
import pandas as pd
from selenium import webdriver

from tqdm import tqdm
import os
from bs4 import BeautifulSoup
import requests
import re

import zipfile
import shutil

import os


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

import json
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()

import psycopg2
from psycopg2 import sql

import sys
sys.path.append("../")  

In [4]:
keywallapop= os.getenv("keywallapop")

Cuidado, de wallapop solo hay 20 llamadas mensuales

In [ ]:

url = "https://wallapop3.p.rapidapi.com/search"

querystring = {"query":"nissan","minPrice":"1000","sort":"mostRecent"}

headers = {
	"x-rapidapi-key": keywallapop,
	"x-rapidapi-host": "wallapop3.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

resposa=response.json()

SOLO NOS INTESESA:  
- id (pasaré a numérico)  
- user_id  
- title  
- description  
- category_id (lo quiero para filtrar por 100(codigo correspondiente a coches)para que no me coja otros códigos que son de carroceria etc)  
- price (que está en euros)  
- location (por si las moscas)  
- capacidad de traslado (no creo que me lo quede)  
- refurbished(igual la mantengo pero los true los paso a 1 y los false a 0 porque en sql existe el bit)

In [19]:
df_crudo=pd.DataFrame(resposa).head(4)

In [10]:
with open("../src", "w") as json_file:
    json.dump(resposa, json_file, indent=4)

In [13]:
len (resposa)

40

In [37]:
lista=[]
df_final=pd.DataFrame()
for i in range(0,40):
    id_wallapop=resposa[i]["id"]
    id_user=resposa[i]["user_id"]
    title=resposa[i]["title"]
    descripcion=resposa[i]["description"]
    id_category=resposa[i]["category_id"]
    precio=resposa[i]["price"]["amount"]
    latitud=resposa[i]["location"]["latitude"]
    longitud=resposa[i]["location"]["longitude"]
    ciudad=resposa[i]["location"]["city"]
    provincia=resposa[i]["location"]["region2"]
    envio_disponible=resposa[i]["shipping"]["item_is_shippable"]
    reparado=resposa[i]["is_refurbished"]["flag"]

    lista.append([id_wallapop,id_user,title,descripcion,id_category,precio,latitud, longitud,ciudad,provincia,envio_disponible,reparado] )
    df=pd.DataFrame(lista)
    df_final= pd.concat([df_final,df])

In [38]:
df_crudo.head()
dicc_encabezados={0:"id_wallapop",
                  1: "id_user",
                  2: "title",
                  3: "descripcion",
                  4: "id_category",
                  5: "precio",
                  6: "latitud",
                  7: "longitud",
                  8: "ciudad",
                  9: "provincia",
                  10: "envio_disponible",
                  11: "reparado"}

In [39]:
df_final.columns= df_final.columns.map(dicc_encabezados)

In [40]:
df_final.sample(3)

,id_wallapop,id_user,title,descripcion,id_category,precio,latitud,longitud,ciudad,provincia,envio_disponible,reparado
11,pzp2xeme9mj3,kp61gov98765,Nissan Qashqai 2016 pegatina C de DGT,vehiculo en perfecto estado general \nitv reci...,100,12900.0,41.458269,2.250253,Badalona,Barcelona,False,False
14,mzn27k7nekzn,4w67vengy96x,carroceria caja nissan cabstar,"2 carrocerías en muy buen estado,para camiones...",12800,1400.0,42.808361,-1.658409,Pamplona/Iruña,Navarra,True,False
20,nz041pxdyvjo,lqzmowoqyx6v,Nissan Patrol 1990,Vendo mi querido Patrol por necesidad. Motor d...,100,2600.0,36.914983,-2.442271,Pechina,Almería,False,False


In [41]:
df_final.shape

(820, 12)

ESTA LISTO PARA LA LIMPIEZA LO DE WALLAPOP

In [28]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 820 entries, 0 to 39
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_wallapop       820 non-null    object 
 1   id_user           820 non-null    object 
 2   title             820 non-null    object 
 3   descripcion       820 non-null    object 
 4   id_category       820 non-null    int64  
 5   precio            820 non-null    float64
 6   latitud           820 non-null    float64
 7   longitud          820 non-null    float64
 8   ciudad            820 non-null    object 
 9   provincia         820 non-null    object 
 10  envio_disponible  820 non-null    bool   
 11  reparado          820 non-null    bool   
dtypes: bool(2), float64(3), int64(1), object(6)
memory usage: 72.1+ KB


In [29]:
keyconcesionario= os.getenv("keyconcesionario")

In [64]:


# Paso 1: Buscar concesionarios Nissan en Madrid
search_url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': f'Bearer {keyconcesionario}',
}
params = {
    'term': 'concesionario Nissan',
    'location': 'Madrid',
    'limit': 50 
}

response = requests.get(search_url, headers=headers, params=params)

if response.status_code == 200:
    businesses = response.json().get('businesses', [])
    lista=[]
    df_final=pd.DataFrame()
    for business in businesses:
        business_id = business['id']
        business_name= business['name']
        calificacion= business['rating']
        url= business["url"]
        lista.append([business_id,business_name,calificacion,url])
        df=pd.DataFrame(lista)
        df_finalc= pd.concat([df_finalc, df])

else:
    print(f"Error en la búsqueda de concesionarios: {response.status_code}")


In [66]:
df_finalc

,0,1,2,3
0,foS71yZRhEOidmM_aVqiwA,Nissan Motor España,1.0,NaN
1,B2cXNS5efEAFgATa28KADA,Quiero Comprar un Coche,5.0,NaN
2,41EVs88iuczCSuauRdMaoQ,Nissan Iberia,0.0,NaN
3,sjQ22MDvy_WOs_nG-YIrLg,Nissan Iberia,0.0,NaN
4,Sjq5F7RT_s59ERD2CzPXKQ,Nissan Iberia,0.0,NaN
...,...,...,...,...
1,B2cXNS5efEAFgATa28KADA,Quiero Comprar un Coche,5.0,https://www.yelp.com/biz/quiero-comprar-un-coc...
2,41EVs88iuczCSuauRdMaoQ,Nissan Iberia,0.0,https://www.yelp.com/biz/nissan-iberia-madrid-...
3,sjQ22MDvy_WOs_nG-YIrLg,Nissan Iberia,0.0,https://www.yelp.com/biz/nissan-iberia-madrid-...
4,Sjq5F7RT_s59ERD2CzPXKQ,Nissan Iberia,0.0,https://www.yelp.com/biz/nissan-iberia-madrid?...


In [100]:
url3= "https://api.nhtsa.gov/SafetyRatings/modelyear/2013/make/nissan/model/micra"
url2= "https://api.nhtsa.gov/SafetyRatings/VehicleId/7520"
response3= requests.get(url3)
responsa3= response3.json()
id_vehicle=responsa3["Results"][0]["VehicleId"]

IndexError: list index out of range

In [ ]:

url2= f"https://api.nhtsa.gov/SafetyRatings/VehicleId/{id_vehicle}"
response2= requests.get(url2)
responsa2= response2.json()
rating_general=responsa2["Results"][0]["OverallRating"]

IndexError: list index out of range

'Not Rated'